In [29]:
import folium
import pandas as pd
import geopandas as gpd

In [32]:
schools = pd.read_csv('geodata/Education_Directory.csv')
schools_hartford = schools [ schools.Town == 'Hartford' ].filter(['School Name', 'Organization Type', 'Location'])

# Clean up location coordinates
schools_hartford.Location = schools_hartford.Location.apply(lambda x: x.split('(')[1][:-1])

# Fix locations known to be wrong
schools_hartford.loc[schools_hartford['School Name'] == 'Parkville Community School', 'Location'] = '41.7566013,-72.7077116'

#schools_hartford['Organization Type'].value_counts()

gpd.GeoDataFrame(schools_hartford)

,School Name,Organization Type,Location
10,M. D. Fox School,Public Schools,"41.746561, -72.680879"
48,Jumoke Academy,Public Charter Schools,"41.771468, -72.697715"
68,Burr School,Public Schools,"41.744899, -72.67309"
80,Achievement First Hartford Academy Inc.,Public Charter Schools,"41.784608, -72.697162"
83,"Breakthrough Magnet School, South",Public Schools,"41.746341, -72.70739"
...,...,...,...
1108,"M. L. King, Jr. School",Public Schools,"41.795753, -72.700253"
1139,HPHS Engineering and Green Technology Academy,Public Schools,"41.765518, -72.700214"
1157,Sport and Medical Sciences Academy,Public Schools,"41.752365, -72.660526"
1197,McDonough Middle School,Public Schools,"41.750413, -72.697811"


In [67]:
# Initialize map
m = folium.Map(
    location=[41.7625, -72.6842],
    tiles=None,
    zoom_start=13,
    control_scale=True,
)

# Add CartoDB Positron baselayer
tiles = folium.TileLayer(
    tiles='CartoDB positron',
    control=False,
    zoom_start=12
)

# Add link to GitHub repo to attribution
tiles.options['attribution'] = '{} | {}'.format(
    '<a href="https://github.com/JackDougherty/school-search-tool">View code on GitHub</a>',
    tiles.options['attribution']
)

tiles.add_to(m)

colors = {
    'Zone 1': 'purple',
    'Zone 2': 'orange',
    'Zone 3': 'yellow',
    'Zone 4': 'lightblue'
}

# School zones
zones = folium.GeoJson(
    f'geodata/hartford-school-attendance-zones-2010.geojson',
    name='School Zones',
    control=False,
    style_function=lambda x: {
        'fillOpacity': 0.4,
        'fillColor': colors[ x['properties']['Name'] ],
        'color': 'black',
        'weight': '1'
    }
).add_to(m)


for index, row in schools_hartford.iterrows():
    folium.Marker(
        location=row.Location.split(','),
        tooltip=row['School Name'],
        icon=folium.Icon(color='lightblue', icon_color='white', icon='graduation-cap', prefix='fa')
    ).add_to(m)

# Add Tooltip data
folium.GeoJsonTooltip(fields=['Name'], aliases=['School Attendance']).add_to(zones)

m.save('index2.html')

## Manual additions

### Geocoder

```html
<link rel="stylesheet" href="https://unpkg.com/leaflet-control-geocoder/dist/Control.Geocoder.css" />
<script src="https://unpkg.com/leaflet-control-geocoder/dist/Control.Geocoder.js"></script>
```

```js
L.Control.geocoder({
  collapsed: false,
  geocoder: L.Control.Geocoder.nominatim({
    geocodingQueryParams: {
      viewbox: '-72.719741,41.720597,-72.640434,41.813167',
      bounded: 1,
    }
  }),
}).addTo(map_3c76f3c5303f44439633e377a081720e);
```

```html
<style>
  .leaflet-right {
    left: calc(50% - 140px);
  }

  .leaflet-control-geocoder {
    float: left !important;
    font-size: 1.3em;
  }

  .leaflet-top.leaflet-right::before {
    content: "Hartford School Zone Lookup";
    font-size: 1.8em;
    font-weight: bold;
    float: left;
    margin-left: -15px;
    padding: 4px;
    border-radius: 4px;
  }

  .leaflet-control-geocoder-icon {
    padding-bottom: 5px;
  }
</style>
```